In [2]:
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from io import StringIO
import scipy
import scipy.stats               # For reciprocal distribution
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer
from sklearn import svm
import sklearn.gaussian_process 
import sklearn.tree        # For DecisionTreeClassifier class
import sklearn.ensemble    # For RandomForestClassifier class
import sklearn.linear_model # For Logistic Classifier
#from sklearn.neighbors import LSHForest
import sklearn.naive_bayes #For Naive Bayes
import sklearn.neural_network #For MLP classifier
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Ignore sklearn deprecation warnings
warnings.filterwarnings("ignore", category=FutureWarning)       # Ignore sklearn deprecation warnings
np.set_printoptions(precision=20, suppress=True)




class student_performance:
    def __init__(self):
        pass
    
    def preprocess_data(self,file):
        f = open(file,"r")
        c = StringIO(f.read())
        print("READING TRAIN DATA")
       
        #Read data for One hot encoding
        X_string_OH = np.char.strip(np.genfromtxt(c,dtype='str',delimiter = ';',usecols = (0,1,3,5,8,9,10,11),skip_header=1))
        f = open(file,"r")
        c = StringIO(f.read())
        
        #Read Data for ordinal encoding
        X_string_OE = np.char.strip(np.genfromtxt(c,dtype='str',delimiter = ';',usecols = (4,15,16,17,18,19,20,21,22),skip_header=1))
        X_float = np.loadtxt(file,delimiter = ";",usecols = (2,6,7,12,13,14,23,24,25,26,27,28,29,30,31,32), dtype = np.float,skiprows=1).astype(int)
        print(X_string_OE.shape)
        print(X_string_OH.shape)
        print(X_float.shape)
        X_soh = self.preprocess_data_OH(X_string_OH)
        X_soe = self.preprocess_data_OE(X_string_OE)
        X = np.column_stack((X_soh,X_soe,X_float[:,:-1]))
        y = X_float[:,-1]
        return(X,y)
    
    def preprocess_data_OH(self,X):
        X=X.tolist()
        encoder = preprocessing.OneHotEncoder(sparse = False)
        print(encoder.fit(X))
        X = encoder.transform(X)
        print(X[0])
        return(X)
    
    def preprocess_data_OE(self,X):
        X=X.tolist()
        encoder = preprocessing.OrdinalEncoder()
        encoder.fit(X)
        X = encoder.transform(X)
        print(X[5])
        return(X)
    
    def scale_data_1(self,X):
        scaler = preprocessing.MinMaxScaler((0,1)).fit(X)
        return(scaler)
    
    def scale_data_5(self,X):
 
        scaler = preprocessing.MinMaxScaler((0,5)).fit(X)
        return(scaler)
    
    def random_CV(self,clf,X,y,param_grid,n_iter,cv):
 
        print("Starting search")
       
        random_search = model_selection.RandomizedSearchCV(clf, param_distributions = param_grid,n_iter = n_iter, cv = cv,
                                           iid = False,verbose=1,n_jobs = 4)
        random_search.fit(X, y)
        print("best parameters:", random_search.best_params_)
        print("%.1f%% accuracy on validation sets (average)" % (random_search.best_score_*100))
        return random_search.best_params_
    
    def SVR_reg(self,X,y):
        print("SVR classifier called")
        svr_clf = svm.SVR()
        param_dist = {
            'C'     : scipy.stats.reciprocal(1.0, 1000.),
            'kernel': ['rbf','poly','linear'],
            'degree': [2],
            'gamma' : scipy.stats.reciprocal(0.01, 10.)
        }
        return self.random_CV(svr_clf,X,y,param_dist,15,3)
    
    def DTR_reg(self,X,y):
        print("Decision Tree Regressor called")
        dt_reg = sklearn.tree.DecisionTreeRegressor()
        param_dist  ={
            "splitter" : ['best', 'random'],
            "max_depth" :[None,100,150,200,250,300],
            "max_features": ["sqrt","log2",None]
        }
        return self.random_CV(dt_reg,X,y,param_dist,10,3)
    
    def RF_reg(self,X,y):
        print("Random Forest Regressor Called")
        rf_reg = sklearn.ensemble.RandomForestRegressor()
        param_dist = {
            "n_estimators" : [10,25,50,75,100,125,150,175,200],
            "max_depth" :[None,100,150,200,250,300],
            "max_features": ["sqrt","log2",None]
        }
        return self.random_CV(rf_reg,X,y,param_dist,12,3)
    
    def ADB_reg(self,X,y):
        print("Ada Boost Regressor called")
        adb_reg = sklearn.ensemble.AdaBoostRegressor()
        param_dist = {
            "n_estimators" : [50,75,100,150,200,250,300,400,500],
            "loss" : ['linear', 'square', 'exponential'],
            "learning_rate" : [0.5,0.75,1,1.5]
        }
        return self.random_CV(adb_reg,X,y,param_dist,10,3)
    
    def GP_reg (self,X,y):
        print("Gaussian Process Regression called")
        
        gp_reg = sklearn.gaussian_process.GaussianProcessRegressor()
        param_dist = {
            'alpha' : scipy.stats.reciprocal(np.exp(-12), np.exp(-1)),
            "normalize_y" : [False, True]
        }
        return self.random_CV(gp_reg,X,y,param_dist,10,3)
    
    def LR_reg(self,X,y):
        print("Linear Regression called")
        
        lr_reg = sklearn.linear_model.LinearRegression()
        param_dist = {
            'fit_intercept' : [True, False],
            'normalize': [True,False]
        }
        return self.random_CV(lr_reg,X,y,param_dist,4,3)
        
    def MLP_reg(self,X,y):
        print("Neural Network regressor Called")
        mlp_reg = sklearn.neural_network.MLPRegressor()
        param_dist = {
            "hidden_layer_sizes" : [(50,), (100,),(50,50),(100,50),(100,100),(50,50,50)],
            "activation" :['tanh', 'relu','logistic','identity'],
            "solver" : ['lbfgs', 'sgd'], #smaller Data set, no adam
            "alpha" : scipy.stats.reciprocal(0.00005,0.1),
            "learning_rate" : ['constant','invscaling'],
            "max_iter" : [100,200,300,500,700]
        }
        return self.random_CV(mlp_reg,X,y,param_dist,15,3)
    
    def train_reg(self,reg,params,X,y):
        reg.set_params(**params)
        reg.fit(X,y)
        print("Complete Training Accuracy")
        print(reg.score(X,y))
        return reg
    
    def start(self):
        print("******Regression of Student Performance Data Set Begins ******")
        file = 'student-por.csv'
        (X,y)=self.preprocess_data(file)
        
        
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
        
        print("Calculating Standard Scaler using training data")
        scaler_1 = self.scale_data_1(X_train[:,:-2])  #Scale all the first 30 attributes to get a val range of 0,1
        scaler_2 = self.scale_data_5(X_train[:,X_train.shape[1]-2:X_train.shape[1]]) #Scale the imp last 2 attributes for a val range 0,5, to show its importance
        
        X_train_1 = scaler_1.transform(X_train[:,:-2])
        X_test_1 = scaler_1.transform(X_test[:,:-2])
        
        X_train_2 = scaler_2.transform(X_train[:,X_train.shape[1]-2:X_train.shape[1]])
        X_test_2 = scaler_2.transform(X_test[:,X_train.shape[1]-2:X_train.shape[1]])

        X_train = np.column_stack((X_train_1,X_train_2))
        X_test = np.column_stack((X_test_1,X_test_2))
        
        print("**Training of Student_performance starts**")
        
        print("__Training SVR__")
        
        svr_reg = self.train_reg(svm.SVR(),self.SVR_reg(X_train,y_train),X_train,y_train)
        
        print("__Training DTR__")
        dt_reg = self.train_reg(sklearn.tree.DecisionTreeRegressor(),self.DTR_reg(X_train,y_train),X_train,y_train)
        
        print("__Training Random Forest Reg__")
        rf_reg = self.train_reg(sklearn.ensemble.RandomForestRegressor(),self.RF_reg(X_train,y_train),X_train,y_train)
        
        print("__Training AdaBoost Reg__")
        adb_reg = self.train_reg(sklearn.ensemble.AdaBoostRegressor(),self.ADB_reg(X_train,y_train),X_train,y_train)
        
        print("__Training Gaussian proc Reg__")
        gp_reg = self.train_reg(sklearn.gaussian_process.GaussianProcessRegressor(),self.GP_reg(X_train,y_train),X_train,y_train)
        
        print("__Training Linear Reg__")
        lin_reg = self.train_reg(sklearn.linear_model.LinearRegression(),self.LR_reg(X_train,y_train),X_train,y_train)
        
        print("__Training Neural Net Reg__")
        nn_reg = self.train_reg(sklearn.neural_network.MLPRegressor(),self.MLP_reg(X_train,y_train),X_train,y_train)
        
        print("**Test Data Prediction Begins*")
        
        
        print("Testing SVM Regressionr")
        print(svr_reg.score(X_test,y_test))
        
        print("Testing Decision Trees")
        print(dt_reg.score(X_test,y_test))
        
        print("Testing Random Forests")
        print(rf_reg.score(X_test,y_test))
        
        print("Testing Adaboost")
        print(adb_reg.score(X_test,y_test))
        
        
        print("Testing Gaussian Process reg")
        print(gp_reg.score(X_test,y_test))
        
        print("Testing Linear Regression")
        print(lin_reg.score(X_test,y_test))
        
        print("Testing Neural Network/MLP")
        print(nn_reg.score(X_test,y_test))
        
        return
    
obj = student_performance()
obj.start()
        


******Regression of Student Performance Data Set Begins ******
READING TRAIN DATA
(649, 9)
(649, 8)
(649, 16)
OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=False)
[1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1.
 0.]
[1. 0. 1. 0. 1. 1. 1. 1. 0.]
Calculating Standard Scaler using training data
**Training of Student_performance starts**
__Training SVR__
SVR classifier called
Starting search
Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:  1.5min finished


best parameters: {'C': 1.2061130238274285, 'degree': 2, 'gamma': 0.01442358591558826, 'kernel': 'linear'}
83.9% accuracy on validation sets (average)
Complete Training Accuracy
0.8543205946687048
__Training DTR__
Decision Tree Regressor called
Starting search
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


best parameters: {'splitter': 'best', 'max_features': None, 'max_depth': 150}
80.9% accuracy on validation sets (average)
Complete Training Accuracy
1.0
__Training Random Forest Reg__
Random Forest Regressor Called
Starting search
Fitting 3 folds for each of 12 candidates, totalling 36 fits


...

best parameters: {'alpha': 6.586039328564779e-05, 'normalize_y': True}
30.3% accuracy on validation sets (average)
Complete Training Accuracy
0.9999999970653035
__Training Linear Reg__
Linear Regression called
Starting search
Fitting 3 folds for each of 4 candidates, totalling 12 fits
best parameters: {'normalize': True, 'fit_intercept': False}
82.6% accuracy on validation sets (average)
Complete Training Accuracy
0.866607013316109
__Training Neural Net Reg__
Neural Network regressor Called
Starting search
Fitting 3 folds for each of 15 candidates, totalling 45 fits

[Parallel(n_jobs=4)]: Done   5 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:   23.7s finished
C:\Users\sadiq\Miniconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


best parameters: {'activation': 'identity', 'alpha': 0.00026143438700713793, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'max_iter': 300, 'solver': 'sgd'}
83.0% accuracy on validation sets (average)
Complete Training Accuracy
0.865135535800553
**Test Data Prediction Begins*
Testing SVM Regressionr
0.8058996892058753
Testing Decision Trees
0.7193980472847575
Testing Random Forests
0.8139252650305664
Testing Adaboost
0.7465255533909212
Testing Gaussian Process reg
0.4170978679196766
Testing Linear Regression
0.8016773948830268
Testing Neural Network/MLP
0.8083536255107628


C:\Users\sadiq\Miniconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [6]:
import numpy as np
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from io import StringIO
import scipy
import scipy.stats               # For reciprocal distribution
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer
from sklearn import svm
import sklearn.gaussian_process 
import sklearn.tree        # For DecisionTreeClassifier class
import sklearn.ensemble    # For RandomForestClassifier class
import sklearn.linear_model # For Logistic Classifier
#from sklearn.neighbors import LSHForest
import sklearn.naive_bayes #For Naive Bayes
import sklearn.neural_network #For MLP classifier
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Ignore sklearn deprecation warnings
warnings.filterwarnings("ignore", category=FutureWarning)       # Ignore sklearn deprecation warnings
np.set_printoptions(precision=20, suppress=True)

class wine:
    
    def __init__(self):
        pass
    
    def SFS(self,X,y): #Feature selection as some features are useless
        size = 11
        cols = [0,1,2,3,4,5,6,7,8,9,10]
        min_score = 0
        useless_col = 20
        useless_cols = []
        rf_reg = sklearn.ensemble.RandomForestRegressor()
        while(len(cols)> 8):
            min_score = 0
            X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
            for i in cols:
                if i  in useless_cols:
                    continue   
                X_sfs = np.column_stack((X_train[:,0:i],X_train[:,(i+1):len(cols)]))
                    
                rf_reg.fit(X_sfs,y_train);
                score = rf_reg.score(np.column_stack((X_test[:,0:i],X_test[:,(i+1):len(cols)])),y_test)
                if(score > min_score):
                    min_score = score
                    print(score)
                    print("removing %d" % i)
                    useless_col = i
            useless_cols.append(useless_col)
            #cols.pop()
            print("deleting")
            print(useless_col)
            np.delete(X,useless_col,1)
        return useless_cols

    
    def scale_data(self,X):
        scaler = preprocessing.StandardScaler().fit(X)
        X = scaler.transform(X)
        return(X,scaler)
    def random_CV(self,clf,X,y,param_grid,n_iter,cv):
 
        print("Starting search")
       
        random_search = model_selection.RandomizedSearchCV(clf, param_distributions = param_grid,n_iter = n_iter, cv = cv,
                                           iid = False,verbose=1,n_jobs = 4)
        random_search.fit(X, y)
        print("best parameters:", random_search.best_params_)
        print("%.1f%% accuracy on validation sets (average)" % (random_search.best_score_*100))
        return random_search.best_params_
    
    def SVR_reg(self,X,y):
        print("SVR classifier called")
        svr_clf = svm.SVR()
        param_dist = {
            'C'     : scipy.stats.reciprocal(1.0, 1000.),
            'kernel': ['rbf','linear'],
            'degree': [2],
            'gamma' : scipy.stats.reciprocal(0.01, 10.)
        }
        return self.random_CV(svr_clf,X,y,param_dist,5,3)
    
    def DTR_reg(self,X,y):
        print("Decision Tree Regressor called")
        dt_reg = sklearn.tree.DecisionTreeRegressor()
        param_dist  ={
            "splitter" : ['best', 'random'],
            "max_depth" :[None,100,150,200,250,300],
            "max_features": ["sqrt","log2",None]
        }
        return self.random_CV(dt_reg,X,y,param_dist,10,3)
    
    def RF_reg(self,X,y):
        print("Random Forest Regressor Called")
        rf_reg = sklearn.ensemble.RandomForestRegressor()
        param_dist = {
            "n_estimators" : [10,25,50,75,100,125,150,175,200],
            "max_depth" :[None,100,150,200,250,300],
            "max_features": ["sqrt","log2",None]
        }
        return self.random_CV(rf_reg,X,y,param_dist,12,3)
    
    def ADB_reg(self,X,y):
        print("Ada Boost Regressor called")
        adb_reg = sklearn.ensemble.AdaBoostRegressor()
        param_dist = {
            "n_estimators" : [50,75,100,150,200,250,300,400,500],
            "loss" : ['linear', 'square', 'exponential'],
            "learning_rate" : [0.5,0.75,1,1.5]
        }
        return self.random_CV(adb_reg,X,y,param_dist,10,3)
    
    def GP_reg (self,X,y):
        print("Gaussian Process Regression called")
        
        gp_reg = sklearn.gaussian_process.GaussianProcessRegressor()
        param_dist = {
            'alpha' : scipy.stats.reciprocal(np.exp(-12), np.exp(-1)),
            "normalize_y" : [False, True]
        }
        return self.random_CV(gp_reg,X,y,param_dist,10,3)
    
    def LR_reg(self,X,y):
        print("Linear Regression called")
        
        lr_reg = sklearn.linear_model.LinearRegression()
        param_dist = {
            'fit_intercept' : [True, False],
            'normalize': [True,False]
        }
        return self.random_CV(lr_reg,X,y,param_dist,4,3)
        
    def MLP_reg(self,X,y):
        print("Neural Network regressor Called")
        mlp_reg = sklearn.neural_network.MLPRegressor()
        param_dist = {
            "hidden_layer_sizes" : [(50,), (100,),(50,50),(100,50),(100,100),(50,50,50)],
            "activation" :['tanh', 'relu','logistic','identity'],
            "solver" : ['lbfgs', 'sgd'], #smaller Data set, no adam
            "alpha" : scipy.stats.reciprocal(0.00005,0.1),
            "learning_rate" : ['constant','invscaling'],
            "max_iter" : [100,200,300,500,700]
        }
        return self.random_CV(mlp_reg,X,y,param_dist,15,3)
    
    def train_reg(self,reg,params,X,y):
        reg.set_params(**params)
        reg.fit(X,y)
        print("Complete Training Accuracy")
        print(reg.score(X,y))
        return reg  
    
    def start(self):
        file = 'winequality-white.csv'
        
        X_float = np.loadtxt(file,delimiter = ";", dtype = np.float,skiprows=1)
        
        print(X_float[0])
        
        X,y = X_float[:,:-1],X_float[:,-1]
        
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.2)
        X_train,scaler = self.scale_data(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        print("Calinng SFS")
     
        print("**Training of Student_performance starts**")
        
        print("__Training SVR__")
        
        #svr_reg = self.train_reg(svm.SVR(),self.SVR_reg(X_train,y_train),X_train,y_train)
        
        print("__Training DTR__")
        dt_reg = self.train_reg(sklearn.tree.DecisionTreeRegressor(),self.DTR_reg(X_train,y_train),X_train,y_train)
        
        print("__Training Random Forest Reg__")
        rf_reg = self.train_reg(sklearn.ensemble.RandomForestRegressor(),self.RF_reg(X_train,y_train),X_train,y_train)
        
        print("__Training AdaBoost Reg__")
        adb_reg = self.train_reg(sklearn.ensemble.AdaBoostRegressor(),self.ADB_reg(X_train,y_train),X_train,y_train)
        
        print("__Training Gaussian proc Reg__")
        gp_reg = self.train_reg(sklearn.gaussian_process.GaussianProcessRegressor(),self.GP_reg(X_train,y_train),X_train,y_train)
        
        print("__Training Linear Reg__")
        lin_reg = self.train_reg(sklearn.linear_model.LinearRegression(),self.LR_reg(X_train,y_train),X_train,y_train)
        
      #  print("__Training Neural Net Reg__")
       # nn_reg = self.train_reg(sklearn.neural_network.MLPRegressor(),self.MLP_reg(X_train.astype(float),y_train),X_train.astype(float),y_train)
        
        print("**Test Data Prediction Begins*")
        
        
        #print("Testing SVM Regressionr")
        #print(svr_reg.score(X_test,y_test))
        
        print("Testing Decision Trees")
        print(dt_reg.score(X_test,y_test))
        
        print("Testing Random Forests")
        print(rf_reg.score(X_test,y_test))
        
        print("Testing Adaboost")
        print(adb_reg.score(X_test,y_test))
        
        
        print("Testing Gaussian Process reg")
        print(gp_reg.score(X_test,y_test))
        
        print("Testing Linear Regression")
        print(lin_reg.score(X_test,y_test))
        
        #print("Testing Neural Network/MLP")
       # print(nn_reg.score(X_test,y_test))
        
        
        return
    
obj = wine()
obj.start()
        

[  7.      0.27    0.36   20.7     0.045  45.    170.      1.001   3.
   0.45    8.8     6.   ]
Calinng SFS
**Training of Student_performance starts**
__Training SVR__
__Training DTR__
Decision Tree Regressor called
Starting search
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    2.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


best parameters: {'splitter': 'best', 'max_features': 'sqrt', 'max_depth': 300}
-2.6% accuracy on validation sets (average)
Complete Training Accuracy
1.0
__Training Random Forest Reg__
Random Forest Regressor Called
Starting search
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=4)]: Done  36 out of  36 | elapsed:   14.1s finished


best parameters: {'n_estimators': 175, 'max_features': 'sqrt', 'max_depth': None}
47.8% accuracy on validation sets (average)
Complete Training Accuracy
0.9354545723463749
__Training AdaBoost Reg__
Ada Boost Regressor called
Starting search
Fitting 3 folds for each of 10 candidates, totalling 30 fits


...

best parameters: {'n_estimators': 500, 'loss': 'linear', 'learning_rate': 0.75}
30.2% accuracy on validation sets (average)
Complete Training Accuracy
0.36494775112583416
__Training Gaussian proc Reg__
Gaussian Process Regression called
Starting search
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:   48.0s finished


best parameters: {'alpha': 0.0016456050780517969, 'normalize_y': True}
19.4% accuracy on validation sets (average)
Complete Training Accuracy
0.9999978878682293
__Training Linear Reg__
Linear Regression called
Starting search
Fitting 3 folds for each of 4 candidates, totalling 12 fits
best parameters: {'normalize': True, 'fit_intercept': True}
27.7% accuracy on validation sets (average)
Complete Training Accuracy
0.28243072204579467
**Test Data Prediction Begins*
Testing Decision Trees
-10.71716651826384
Testing Random Forests
-0.0007348976535195284
Testing Adaboost
-0.047367614796532516
Testing Gaussian Process reg


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:    0.0s finished


-0.008666626632592322
Testing Linear Regression
-45.64090440877667
